In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 49.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=44f883e31bdcef32f938367d0dcf0d2f41347d365b026c9368106d0948f3282c
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.python.pyspark.shell import spark
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

df = spark.read.options(delimiter=',', header=True).csv('/content/bank.csv')
df = df.withColumn("AG", df["age"] - 0).withColumn("BL", df["balance"] - 0).\
    withColumn("DU", df["duration"] - 0).withColumn("CA", df["campaign"] - 0).withColumn("PD", df["pdays"] - 0).withColumn("PR", df["previous"] - 0)
df.show(5)

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.1
      /_/

Using Python version 3.7.10 (default, Feb 20 2021 21:17:23)
Spark context Web UI available at http://92b7f2778966:4040
Spark context available as 'sc' (master = local[*], app id = local-1618888733847).
SparkSession available as 'spark'.
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+----+------+------+---+----+---+
|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|  AG|    BL|    DU| CA|  PD| PR|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+----+------+------+---+----+---+
| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unkno

In [ ]:
df = df.select('age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit'
, 'AG', 'BL', 'DU', 'CA', 'PD', 'PR')
cols = df.columns
df.show(5)

+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+----+------+------+---+----+---+
|age|       job|marital|education|default|balance|housing|loan|contact|duration|campaign|pdays|previous|poutcome|deposit|  AG|    BL|    DU| CA|  PD| PR|
+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+----+------+------+---+----+---+
| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|    1042|       1|   -1|       0| unknown|    yes|59.0|2343.0|1042.0|1.0|-1.0|0.0|
| 56|    admin.|married|secondary|     no|     45|     no|  no|unknown|    1467|       1|   -1|       0| unknown|    yes|56.0|  45.0|1467.0|1.0|-1.0|0.0|
| 41|technician|married|secondary|     no|   1270|    yes|  no|unknown|    1389|       1|   -1|       0| unknown|    yes|41.0|1270.0|1389.0|1.0|-1.0|0.0|
| 55|  services|married|secondary|     no|   2476|    yes|  no|unknown|     

In [ ]:
categoricalColumns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol='indexed' + categoricalCol)
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
labelIndexerFrequency = StringIndexer(inputCol = 'deposit', outputCol = 'label')
stages += [labelIndexerFrequency]
numericCols = ['AG', 'BL', 'DU', 'CA', 'PD', 'PR']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
from pyspark.ml import Pipeline

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.show(5)

+-----+--------------------+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+----+------+------+---+----+---+
|label|            features|age|       job|marital|education|default|balance|housing|loan|contact|duration|campaign|pdays|previous|poutcome|deposit|  AG|    BL|    DU| CA|  PD| PR|
+-----+--------------------+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+----+------+------+---+----+---+
|  1.0|(30,[3,11,13,16,1...| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|    1042|       1|   -1|       0| unknown|    yes|59.0|2343.0|1042.0|1.0|-1.0|0.0|
|  1.0|(30,[3,11,13,16,1...| 56|    admin.|married|secondary|     no|     45|     no|  no|unknown|    1467|       1|   -1|       0| unknown|    yes|56.0|  45.0|1467.0|1.0|-1.0|0.0|
|  1.0|(30,[2,11,13,16,1...| 41|technician|married|secondary|     no|   1270|    yes|  no|unkno

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier
# split to 70-30
(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=2021)
# Train a Gradient Boosted Tree model
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
# Train model. This also runs the indexers.
model = gbt.fit(trainingData)
# Make predictions.
predictions = model.transform(testData)
# Select example rows to display.
predictions.select('age', 'job', 'label', 'prediction', 'probability').show(5)
evaluator = BinaryClassificationEvaluator()
accuracy = evaluator.evaluate(predictions)
print("Gradient Boosted Tree - Test Accuracy = %g" % (accuracy))
print("Gradient Boosted Tree - Test Error = %g" % (1.0 - accuracy))

+---+----------+-----+----------+--------------------+
|age|       job|label|prediction|         probability|
+---+----------+-----+----------+--------------------+
| 35|management|  0.0|       0.0|[0.51778009602793...|
| 37|management|  0.0|       0.0|[0.86696819810124...|
| 42|management|  0.0|       0.0|[0.92164589827077...|
| 42|management|  0.0|       1.0|[0.42031485907003...|
| 51|management|  0.0|       0.0|[0.91377369017576...|
+---+----------+-----+----------+--------------------+
only showing top 5 rows

Gradient Boosted Tree - Test Accuracy = 0.884111
Gradient Boosted Tree - Test Error = 0.115889


In [ ]:
# save model
model.save('/content/myModel')